In [ ]:
import numpy as np

random = np.random.default_rng(seed=42)

In [ ]:
import scipy

from itertools import combinations
import time

# Instance Generator

In [ ]:
def generate_instance(num_items: int,
                      weight_dimension: int=1, 
                      max_weight: int=10e5, 
                      max_profit: int=10e5,
                     ):
    """Generate a knapsack problem instance with weights and
    profits generated uniformly at random without correlation.
    """
    
    weights = random.integers(1, max_weight, size=(num_items, weight_dimension))
    profits = random.integers(0, max_profit, size=(num_items))
    
    max_capacities = 0.75 * np.sum(weights, axis=0)
    min_capacities = 0.50 * np.sum(weights, axis=0)
    
    capacities = random.integers(min_capacities, max_capacities, size=(weight_dimension))
    
    return weights, profits, capacities


# Solution Algorithms

In [ ]:
def solve_exact_ilp(weights, 
                    profits, 
                    capacities
                   ):
    """Use the scipy ILP solver to obtain an exact solution
    to the given knapsack problem.
    """
    
    c = -(profits.T)
    A_ub = weights.T
    b_ub = capacities.T
    
    bounds = (0, 1)
    integrality = 1
    
    start_time = time.perf_counter()
    
    result = scipy.optimize.linprog(c=c, A_ub=A_ub, b_ub=b_ub, 
                                    bounds=bounds, integrality=integrality)
    
    end_time = time.perf_counter()
    
    if not result.success:
        raise ValueError
        
    assert np.all(result.x.dot(weights) < capacities)
    assert round(result.x.dot(profits)) == -round(result.fun)
    
    return -round(result.fun), result.x.astype(int), (end_time - start_time) * 1000


In [ ]:
def solve_exact_enumeration(weights, 
                            profits, 
                            capacities
                           ):
    """A naive exhaustive enumeration baseline to 
    obtain an exact solution to the given knapsack 
    problem.
    """
    
    max_profit = 0
    max_profit_indices = None

    start_time = time.perf_counter()
    
    for m in range(1, len(weights) + 1):
        for indices_set in combinations(range(len(weights)), m):
            local_profit = np.sum(profits[list(indices_set)], axis=0)
            local_weight = np.sum(weights[list(indices_set)], axis=0)

            if np.all(local_weight < capacities) and local_profit > max_profit:
                max_profit = local_profit
                max_profit_indices = list(indices_set)
                
    end_time = time.perf_counter()

    out_indices = np.zeros(len(weights), dtype=int)
    out_indices[max_profit_indices] = 1

    return max_profit, out_indices, (end_time - start_time) * 1000


In [ ]:
def solve_approximate_pech(weights, 
                           profits, 
                           capacities):
    """A greedy-like algorithm to obtain an approximate 
    solution to the given knapsack problem.
    
    See https://www.math.wsu.edu/math/faculty/lih/MDKP.pdf for details.
    """ 

    ## STEP 1
    xj = np.zeros(weights.shape[0])
    E = np.ones(weights.shape[0])
    bi = capacities.copy()
    uj = np.ones(weights.shape[0])
    
    start_time = time.perf_counter()
    
    while True:
        ## STEP 2
        yj = np.min(np.floor(bi / weights), axis=1) * E

        if np.sum(yj) == 0:
            break

        ## STEP 3
        jstar = np.argmax(yj * profits.T)

        ## STEP 4
        ## (Original paper compares with a weighted
        ##  alternative to allow copies of items, but 
        ##  we do not allow repetition of items here)
        yjstar = min((uj[jstar], 1))

        ## STEP 5
        xj[jstar] += yjstar
        bi -= (weights[jstar] * yjstar).astype(int)
        uj[jstar] -= yjstar
        if uj[jstar] == 0:
            E[jstar] = 0

        if np.sum(E) == 0:
            break
            
    end_time = time.perf_counter()
            
    return int(profits.T.dot(xj)), xj.astype(int), (end_time - start_time) * 1000


In [ ]:
def heuristic_greedy(current_weights, 
                     current_profits, 
                     current_capacities
                    ):
    """Bounding heuristic using the greedy algotihm above.
    """
    
    estimated_bound, vec, _ = solve_approximate_pech(current_weights, current_profits, current_capacities)
    return estimated_bound, vec


def heuristic_total_value(current_weights, 
                          current_profits, 
                          current_capacities
                         ):
    """Bounding heuristic that simply returns
    the total profit of all un-used items.
    """
    
    estimated_bound = np.sum(current_profits)
    vec = np.ones(current_weights.shape[0])
    return estimated_bound, vec


In [ ]:
class BranchAndBoundSolver:
    def __init__(self, weights, profits, capacities, 
                 heuristic, 
                 expansion_rule='depth-first',
                 branch_selection_rule='ordered',
                 cache_heuristic_results=False
                ):
        
        self.weights = weights
        self.profits = profits
        self.capacities = capacities
        
        self.lower_bound = 0
        self.heuristic = heuristic
        self.best_solution = None
        
        initial_vector = -np.ones(len(weights), dtype=int)
        _cache_index = (hash((initial_vector ==  1).data.tobytes()),
                        hash((initial_vector == -1).data.tobytes()))
        self.solution_vector_store = [(initial_vector, -1, _cache_index)]
        self.examined_solution_vector_store = set()
        
        self._total_branched = 0
        
        self.expansion_rule = expansion_rule
        self.branch_selection_rule = branch_selection_rule
        
        self._cache_heuristic_results = cache_heuristic_results
        self._heuristic_cache = {}
        
        
    def update_and_check_against_bounds(self, 
                                        potential_solution_vector
                                       ):
        
        _cache_index = (hash((potential_solution_vector ==  1).data.tobytes()),
                        hash((potential_solution_vector == -1).data.tobytes()))
        
        if _cache_index in self.examined_solution_vector_store:
            return False, None, _cache_index
        
        ## All un-branched variables are ignored
        ## in the solution at the current node
        clipped_solution_vector = np.clip(potential_solution_vector, a_min=0, a_max=1)
        current_profit = clipped_solution_vector.dot(self.profits)
        current_weight = clipped_solution_vector.dot(self.weights)
        
        remaining_capacity = self.capacities - current_weight
        
        ## Infeasible node
        if np.any(remaining_capacity < 0):
            return False, None, _cache_index
        
        ## Mask out all variables already branched on
        mask = (potential_solution_vector + 1).astype(bool)
        
        ## New best solution
        if current_profit >= self.lower_bound:
            self.lower_bound = current_profit
            self.best_solution = (current_profit, clipped_solution_vector)
        
        ## If this represents a true leaf node (all variables have been
        ## branched on) or all remaining options would exceed the remaining 
        ## capacity, we will not (or rather, cannot) explore further
        if np.all(mask) or np.all(np.any(self.weights[~mask] > remaining_capacity, axis=-1)):
            return False, None, _cache_index
        
        ## Otherwise we use the heuristic to determine if possible solutions
        ## on the un-branched variables could push the running profit along 
        ## this branch higher than the current lower bound
        if not self._cache_heuristic_results:
            estimated_upper_bound = self.heuristic(self.weights[~mask], 
                                                   self.profits[~mask], 
                                                   remaining_capacity)[0] + current_profit
        else:
            try:
                estimated_upper_bound = self._heuristic_cache[_cache_index]

            except KeyError:
                estimated_upper_bound, heuristic_solution = self.heuristic(self.weights[~mask], 
                                                                           self.profits[~mask], 
                                                                           remaining_capacity)
                estimated_upper_bound += current_profit

                ## In the greedy heuristic case, greedy estimations of appropriate
                ## sub-problems will have the same solution - we cache all these
                ## possible cases to avoid re-computing when it is not necessary
                ## (This would have to be changed for non-greedy heuristics)
                unbranched_indices = (~mask).nonzero()[0]
                p = potential_solution_vector.copy()
                for r in range(len(heuristic_solution) + 1):
                    for c in combinations(np.arange(len(heuristic_solution)), r):
                        c = list(c)
                        potential_solution_vector[unbranched_indices[c]] = heuristic_solution[c]
                        _tmp_cache_index = (hash((potential_solution_vector ==  1).data.tobytes()),
                                            hash((potential_solution_vector == -1).data.tobytes()))
                        self._heuristic_cache[_tmp_cache_index] = estimated_upper_bound
                        potential_solution_vector = p
            
        return estimated_upper_bound > self.lower_bound, estimated_upper_bound, _cache_index
    
    
    def add_to_solution_vector_store(self, 
                                     solution_vector, 
                                     estimated_bound, 
                                     _cache_index
                                    ):
        ## Store acts as stack
        if self.expansion_rule == 'depth-first':
            self.solution_vector_store.append((solution_vector.copy(), estimated_bound, _cache_index))
            
        ## Store acts as queue
        elif self.expansion_rule == 'breadth-first':
            self.solution_vector_store.insert(0, (solution_vector.copy(), estimated_bound, _cache_index))

        ## Store is sorted
        elif self.expansion_rule == 'best-bound':
            for i, (_, bound, _) in enumerate(self.solution_vector_store):
                if estimated_bound >= bound:
                    self.solution_vector_store.insert(i + 1, (solution_vector.copy(), estimated_bound, _cache_index))
                    break
            else:
                self.solution_vector_store.insert(0, (solution_vector.copy(), estimated_bound, _cache_index))
                
        else:
            raise NotImplementedError
            
    
    def solve(self):
        
        while len(self.solution_vector_store) > 0:
            
            current_solution_vector, _, _cache_index = self.solution_vector_store.pop()
            self._total_branched += 1
            self.examined_solution_vector_store.add(_cache_index)
            
            if self.branch_selection_rule == 'ordered':
                i = (current_solution_vector == -1).nonzero()[0][0]

            elif self.branch_selection_rule == 'random':
                i = random.choice((current_solution_vector == -1).nonzero()[0])

            elif self.branch_selection_rule == 'greedy':
                mask = ~((current_solution_vector + 1).astype(bool))
                eff = np.min(np.floor(self.capacities / self.weights), axis=1) * self.profits
                i = np.arange(self.profits.shape[0])[mask][np.argmax(eff[mask])] #self.profits[mask])]
                
            else:
                raise NotImplementedError
                
            ## Check if the branch including the
            ## selected item is worth expanding to
            current_solution_vector[i] = 1
            explore, estimated_bound, _cache_index = self.update_and_check_against_bounds(current_solution_vector)
            
            if explore:
                self.add_to_solution_vector_store(current_solution_vector, estimated_bound, _cache_index)
            else:
                self.examined_solution_vector_store.add(_cache_index)

            ## Check if the branch not including the
            ## selected item is worth expanding to
            current_solution_vector[i] = 0
            explore, estimated_bound, _cache_index = self.update_and_check_against_bounds(current_solution_vector)
            
            if explore:
                self.add_to_solution_vector_store(current_solution_vector, estimated_bound, _cache_index)
            else:
                self.examined_solution_vector_store.add(_cache_index)
            
        return self._total_branched
        
            
    
def solve_approximate_branchandbound(weights, profits, capacities, 
                                     heuristic, 
                                     expansion_rule='depth-first',
                                     branch_selection_rule='ordered',
                                     cache_heuristic_results=False):
    
    ## Wraps a BranchAndBoundSolver with timing functions
        
    solver = BranchAndBoundSolver(weights, profits, capacities, 
                                  heuristic, 
                                  expansion_rule, branch_selection_rule,
                                  cache_heuristic_results)
        
    start_time = time.perf_counter()
    total_branched = solver.solve()
    end_time = time.perf_counter()
    
    return solver.best_solution[0], solver.best_solution[1], (end_time - start_time) * 1000, total_branched
        

# Experiments

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

## Branch & Bound Comparisons

### Variable Ordering

In [ ]:
num_trials = 5
capacity_dimensionality = 5

num_items = [20, 25, 30, 35]

ap_bnb_dfs_ord_hgr_results = []
ap_bnb_dfs_rnd_hgr_results = []
ap_bnb_dfs_grd_hgr_results = []

for i, N in enumerate(num_items):

    print(N)
    
    ap_bnb_dfs_ord_hgr_res = []
    ap_bnb_dfs_rnd_hgr_res = []
    ap_bnb_dfs_grd_hgr_res = []
    
    ## Maximum number of branching actions
    ## (i.e., cases where the algorithm
    ## examines its possible sub-nodes)
    actual_maximum = (2**(N - 1)) - 1
    
    for _ in range(num_trials):
        weights, profits, capacities = generate_instance(N, capacity_dimensionality)

        profit, _, runtime, total_branched = solve_approximate_branchandbound(weights, profits, capacities, 
                                                                              heuristic_greedy, 
                                                                              expansion_rule='depth-first',
                                                                              branch_selection_rule='ordered')
        ap_bnb_dfs_ord_hgr_res.append((total_branched / actual_maximum, runtime, 
                                      (i / (len(num_items) - 1)), N))
        
        
        profit, _, runtime, total_branched = solve_approximate_branchandbound(weights, profits, capacities, 
                                                                              heuristic_greedy, 
                                                                              expansion_rule='depth-first',
                                                                              branch_selection_rule='random')
        ap_bnb_dfs_rnd_hgr_res.append((total_branched / actual_maximum, runtime, 
                                      (i / (len(num_items) - 1)), N))
        
        profit, _, runtime, total_branched = solve_approximate_branchandbound(weights, profits, capacities, 
                                                                              heuristic_greedy, 
                                                                              expansion_rule='depth-first',
                                                                              branch_selection_rule='greedy')
        ap_bnb_dfs_grd_hgr_res.append((total_branched / actual_maximum, runtime, 
                                      (i / (len(num_items) - 1)), N))
        
        
    ap_bnb_dfs_ord_hgr_results += ap_bnb_dfs_ord_hgr_res
    ap_bnb_dfs_rnd_hgr_results += ap_bnb_dfs_rnd_hgr_res
    ap_bnb_dfs_grd_hgr_results += ap_bnb_dfs_grd_hgr_res
    

In [ ]:
cmap = matplotlib.colormaps.get_cmap('brg')

fig, ax = plt.subplots(figsize=(8,5))

def plot(results, marker, fill=True, label=None):
    if fill:
        cm = ax.scatter([r[1] for r in results], [r[0] * 100 for r in results], 
                        color=[cmap(r[2]) for r in results], 
                        marker=marker, label=label)
    else:
        cm = ax.scatter([r[1] for r in results], [r[0] * 100 for r in results], 
                        color=(0,0,0,0), edgecolors=[cmap(r[2]) for r in results], 
                        marker=marker, label=label)
    return cm

cm = plot(ap_bnb_dfs_ord_hgr_results, marker="^", fill=False, label="Ord. Var. Sel., Depth-First, Greedy Heur.")
cm = plot(ap_bnb_dfs_rnd_hgr_results, marker="o", fill=False,  label="Rand. Var. Sel., Depth-First, Greedy Heur.")
cm = plot(ap_bnb_dfs_grd_hgr_results, marker="s", fill=False,  label="Greedy Var. Sel., Depth-First, Greedy Heur.")

ax.set_xscale('log')

ax.set_xlabel('Runtime (ms)')
ax.set_ylabel('% of Maximum Branches')

fmt = '%.2f%%'
yticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(yticks)

cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=cmap), 
                    ticks=np.linspace(0, 1, num=len(num_items)),
                    label='Number of Items')
cbar.ax.set_yticklabels(num_items)

ax.set_title('Weight/Capacity Dimension: {}'.format(capacity_dimensionality))
ax.legend()

plt.show();

### Expansion Rules

In [ ]:
num_trials = 5
capacity_dimensionality = 5

num_items = [10, 15, 20]

ap_bnb_dfs_ord_hgr_results = []
ap_bnb_bfs_ord_hgr_results = []
ap_bnb_bst_ord_hgr_results = []

for i, N in enumerate(num_items):

    print(N)
    
    ap_bnb_dfs_ord_hgr_res = []
    ap_bnb_bfs_ord_hgr_res = []
    ap_bnb_bst_ord_hgr_res = []
    
    ## Maximum number of branching actions
    ## (i.e., cases where the algorithm
    ## examines its possible sub-nodes)
    actual_maximum = (2**(N - 1)) - 1
    
    for _ in range(num_trials):
        weights, profits, capacities = generate_instance(N, capacity_dimensionality)

        profit, _, runtime, total_branched = solve_approximate_branchandbound(weights, profits, capacities, 
                                                                              heuristic_greedy, 
                                                                              expansion_rule='depth-first',
                                                                              branch_selection_rule='ordered')
        ap_bnb_dfs_ord_hgr_res.append((total_branched / actual_maximum, runtime, 
                                      (i / (len(num_items) - 1)), N))
        
        
        profit, _, runtime, total_branched = solve_approximate_branchandbound(weights, profits, capacities, 
                                                                              heuristic_greedy, 
                                                                              expansion_rule='breadth-first',
                                                                              branch_selection_rule='ordered')
        ap_bnb_bfs_ord_hgr_res.append((total_branched / actual_maximum, runtime, 
                                      (i / (len(num_items) - 1)), N))
        
        profit, _, runtime, total_branched = solve_approximate_branchandbound(weights, profits, capacities, 
                                                                              heuristic_greedy, 
                                                                              expansion_rule='best-bound',
                                                                              branch_selection_rule='ordered')
        ap_bnb_bst_ord_hgr_res.append((total_branched / actual_maximum, runtime, 
                                      (i / (len(num_items) - 1)), N))
        
        
    ap_bnb_dfs_ord_hgr_results += ap_bnb_dfs_ord_hgr_res
    ap_bnb_bfs_ord_hgr_results += ap_bnb_bfs_ord_hgr_res
    ap_bnb_bst_ord_hgr_results += ap_bnb_bst_ord_hgr_res
    

In [ ]:
cmap = matplotlib.colormaps.get_cmap('brg')

fig, ax = plt.subplots(figsize=(8,5))

def plot(results, marker, fill=True, label=None):
    if fill:
        cm = ax.scatter([r[1] for r in results], [r[0] * 100 for r in results], 
                        color=[cmap(r[2]) for r in results], 
                        marker=marker, label=label)
    else:
        cm = ax.scatter([r[1] for r in results], [r[0] * 100 for r in results], 
                        color=(0,0,0,0), edgecolors=[cmap(r[2]) for r in results], 
                        marker=marker, label=label)
    return cm

cm = plot(ap_bnb_dfs_ord_hgr_results, marker="^", fill=False, label="Depth-First, Greedy Heur., Ord. Var. Sel.")
cm = plot(ap_bnb_bfs_ord_hgr_results, marker="o", fill=False,  label="Breadth-First, Greedy Heur., Ord. Var. Sel.")
cm = plot(ap_bnb_bst_ord_hgr_results, marker="s", fill=False,  label="Best Bound, Greedy Heur., Ord. Var. Sel.")

ax.set_xscale('log')

ax.set_xlabel('Runtime (ms)')
ax.set_ylabel('% of Maximum Branches')

fmt = '%.2f%%'
yticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(yticks)

cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=cmap), 
                    ticks=np.linspace(0, 1, num=len(num_items)),
                    label='Number of Items')
cbar.ax.set_yticklabels(num_items)

ax.set_title('Weight/Capacity Dimension: {}'.format(capacity_dimensionality))
ax.legend()

plt.show();

### Bounding Heuristic

In [ ]:
num_trials = 5
capacity_dimensionality = 5

num_items = [10, 15, 20]

ap_bnb_dfs_ord_hgr_results = []
ap_bnb_dfs_ord_htv_results = []

for i, N in enumerate(num_items):

    print(N)
    
    ap_bnb_dfs_ord_hgr_res = []
    ap_bnb_dfs_ord_htv_res = []
    
    ## Maximum number of branching actions
    ## (i.e., cases where the algorithm
    ## examines its possible sub-nodes)
    actual_maximum = (2**(N - 1)) - 1
    
    for _ in range(num_trials):
        weights, profits, capacities = generate_instance(N, capacity_dimensionality)
        
        profit, _, runtime, total_branched = solve_approximate_branchandbound(weights, profits, capacities, 
                                                                              heuristic_greedy, 
                                                                              expansion_rule='depth-first',
                                                                              branch_selection_rule='ordered')
        ap_bnb_dfs_ord_hgr_res.append((total_branched / actual_maximum, runtime, 
                                      (i / (len(num_items) - 1)), N))
        
        
        profit, _, runtime, total_branched = solve_approximate_branchandbound(weights, profits, capacities, 
                                                                              heuristic_total_value, 
                                                                              expansion_rule='depth-first',
                                                                              branch_selection_rule='ordered')
        ap_bnb_dfs_ord_htv_res.append((total_branched / actual_maximum, runtime, 
                                      (i / (len(num_items) - 1)), N))
        
        
    ap_bnb_dfs_ord_hgr_results += ap_bnb_dfs_ord_hgr_res
    ap_bnb_dfs_ord_htv_results += ap_bnb_dfs_ord_htv_res
    

In [ ]:
cmap = matplotlib.colormaps.get_cmap('brg')

fig, ax = plt.subplots(figsize=(8,5))

def plot(results, marker, fill=True, label=None):
    if fill:
        cm = ax.scatter([r[1] for r in results], [r[0] * 100 for r in results], 
                        color=[cmap(r[2]) for r in results], 
                        marker=marker, label=label)
    else:
        cm = ax.scatter([r[1] for r in results], [r[0] * 100 for r in results], 
                        color=(0,0,0,0), edgecolors=[cmap(r[2]) for r in results], 
                        marker=marker, label=label)
    return cm

cm = plot(ap_bnb_dfs_ord_hgr_results, marker="^", fill=False, label="Greedy Heur., Depth-First, Ord. Var. Sel.")
cm = plot(ap_bnb_dfs_ord_htv_results, marker="o", fill=False, label="Tot. Val. Heur., Depth-First, Ord. Var. Sel.")

ax.set_xscale('log')

ax.set_xlabel('Runtime (ms)')
ax.set_ylabel('% of Maximum Branches')

fmt = '%.2f%%'
yticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(yticks)

cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=cmap), 
                    ticks=np.linspace(0, 1, num=len(num_items)),
                    label='Number of Items')
cbar.ax.set_yticklabels(num_items)

ax.set_title('Weight/Capacity Dimension: {}'.format(capacity_dimensionality))
ax.legend()

plt.show();

## ILP and PECH and Branch & Bound

In [ ]:
num_trials = 5
capacity_dimensionality = 5

num_items = [30, 40, 50]

exact_ilp_results = []
apprx_pech_results = []
apprx_bnb_results = []

for i, N in enumerate(num_items):
    
    print(N)
    
    ex_ilp_res = []
    ap_pch_res = []
    ap_bnb_res = []
    
    for _ in range(num_trials):
        weights, profits, capacities = generate_instance(N, capacity_dimensionality)
        
        profit, _, runtime = solve_exact_ilp(weights, profits, capacities)
        actual_maximum = profit
        ex_ilp_res.append((profit / actual_maximum, runtime, 
                           (i / (len(num_items) - 1)), N))
        
        profit, _, runtime = solve_approximate_pech(weights, profits, capacities)
        ap_pch_res.append((profit / actual_maximum, runtime, 
                           (i / (len(num_items) - 1)), N))
        
        profit, _, runtime, total_branched = solve_approximate_branchandbound(weights, profits, capacities, 
                                                                              heuristic_greedy, 
                                                                              expansion_rule='depth-first',
                                                                              branch_selection_rule='ordered')
        ap_bnb_res.append((profit / actual_maximum, runtime, 
                           (i / (len(num_items) - 1)), N))
    
    exact_ilp_results += ex_ilp_res
    apprx_pech_results += ap_pch_res
    apprx_bnb_results += ap_bnb_res
    

In [ ]:
cmap = matplotlib.colormaps.get_cmap('brg')

fig, ax = plt.subplots(figsize=(8,5))

def plot(results, marker, fill=True, label=None):
    if fill:
        cm = ax.scatter([r[1] for r in results], [r[0] * 100 for r in results], 
                        color=[cmap(r[2]) for r in results], 
                        marker=marker, label=label)
    else:
        cm = ax.scatter([r[1] for r in results], [r[0] * 100 for r in results], 
                        color=(0,0,0,0), edgecolors=[cmap(r[2]) for r in results], 
                        marker=marker, label=label)
    return cm

cm = plot(exact_ilp_results, marker="o", fill=False, label="ILP (Baseline)")
cm = plot(apprx_pech_results, marker="s", label="PECH (Greedy)")
cm = plot(apprx_bnb_results, marker="^", label="Branch & Bound")

ax.set_xscale('log')

ax.set_xlabel('Runtime (ms)')
ax.set_ylabel('% of Actual Maximum')

fmt = '%.2f%%'
yticks = mtick.FormatStrFormatter(fmt)
ax.yaxis.set_major_formatter(yticks)

cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=cmap), 
                    ticks=np.linspace(0, 1, num=len(num_items)),
                    label='Number of Items')
cbar.ax.set_yticklabels(num_items)

ax.set_title('Weight/Capacity Dimension: {}'.format(capacity_dimensionality))
ax.legend()

plt.show();

-------------